###Data Dictionary

In [1]:
import pandas as pd
raw = pd.read_csv("synthetic_marketplace_daily_raw.csv", parse_dates=["date"])
clean = pd.read_csv("synthetic_marketplace_daily_clean.csv", parse_dates=["date"])

print("Raw rows: ", len(raw), "| Clean rows:", len(clean))
clean.head()

Raw rows:  50400 | Clean rows: 50400


,date,product_id,vendor_id,category,sub_category,price_usd,discount_rate,ad_spend_usd,views,orders,gross_revenue_usd,returns,rating,rating_count,stock_units,avg_fulfillment_days,conversion_rate,return_rate,net_revenue_usd
0,2025-09-01,P00001,V026,Grocery,Fresh,7.48,0.0,0.0,66,6,44.88,0,3.93,852,374,3.1,0.090909,0.000000,44.88
1,2025-09-01,P00002,V010,Home,Furniture,45.19,0.0,0.0,64,10,451.90,0,4.44,2069,503,3.0,0.156250,0.000000,451.90
2,2025-09-01,P00003,V003,Automotive,Parts,40.66,0.0,0.0,41,1,40.66,0,3.99,1451,258,3.5,0.024390,0.000000,40.66
3,2025-09-01,P00004,V004,Kids,Baby Care,21.59,0.0,0.0,51,6,129.54,1,3.93,941,508,5.1,0.117647,0.166667,107.95
4,2025-09-01,P00005,V016,Fashion,Men,25.33,0.0,0.0,54,1,25.33,0,3.67,63,260,6.8,0.018519,0.000000,25.33


###Vendor Performance analysis

In [3]:
from re import A
import pandas as pd

clean = pd.read_csv("synthetic_marketplace_daily_clean.csv", parse_dates=["date"])
vendors = pd.read_csv("vendors_master.csv")

vendor_perf = clean.groupby("vendor_id").agg(
    views = ("views", "sum"),
    orders = ("orders", "sum"),
    net_revenue_usd = ("net_revenue_usd", "sum"),
    ad_spend_usd = ("ad_spend_usd", "sum"),
    returns = ("returns", "sum"),
    avg_discount = ("discount_rate", "mean"),
    avg_fulfillment_days = ("avg_fulfillment_days", "mean"),
    avg_rating = ("rating", "mean"),
).reset_index()

vendor_perf["conversion_rate"] = vendor_perf["orders"]/vendor_perf["views"].replace(0, pd.NA)
vendor_perf["return_rate"] = vendor_perf["returns"]/vendor_perf["orders"].replace(0, pd.NA)
vendor_perf = vendor_perf.merge(vendors, on = "vendor_id", how = "left")

top_rev = vendor_perf.sort_values("net_revenue_usd", ascending = False).head(10)
top_conv = vendor_perf[vendor_perf["views"]>2000].sort_values("conversion_rate", ascending= False).head(10)

top_rev

,vendor_id,views,orders,net_revenue_usd,ad_spend_usd,returns,avg_discount,avg_fulfillment_days,avg_rating,conversion_rate,return_rate,vendor_tier,vendor_region,vendor_quality_score
11,V012,99632,13074,583997.793885,1585.98,462,0.026350,2.908833,4.014667,0.131223,0.035337,Silver,GCC,0.778
4,V005,123885,19929,522346.945392,1397.20,1024,0.027867,3.763333,4.348235,0.160867,0.051382,Gold,GCC,-1.951
0,V001,95597,12461,485641.483071,1222.53,462,0.025182,3.115893,3.995714,0.130349,0.037076,Silver,Levant,0.304
15,V016,95251,9680,407711.706847,1268.60,543,0.025045,3.803278,3.994667,0.101626,0.056095,Silver,GCC,-0.859
8,V009,99781,12231,407620.923752,1312.89,694,0.030873,3.416222,3.925333,0.122578,0.056741,Silver,Asia,-0.017
32,V033,120883,10398,377744.077312,1551.81,950,0.026299,4.172546,4.047222,0.086017,0.091364,Gold,North Africa,1.041
28,V029,94888,14266,369475.463586,1100.60,658,0.028430,3.230536,4.170714,0.150346,0.046124,Gold,Europe,-0.102
29,V030,103882,12605,363777.447140,1153.64,689,0.026915,3.986222,3.958667,0.121340,0.054661,Silver,Levant,0.267
9,V010,93509,9876,362724.520787,1285.75,777,0.029771,4.089359,4.287692,0.105616,0.078676,Silver,Levant,-0.853
18,V019,93648,9018,361337.027115,1404.03,549,0.027057,3.412738,4.275714,0.096297,0.060878,Gold,GCC,0.050


###Product and Demand Analysis

In [4]:
import pandas as pd

clean = pd.read_csv("synthetic_marketplace_daily_clean.csv", parse_dates=["date"])
prod_perf = clean.groupby(["product_id","vendor_id","category","sub_category","price_usd"]).agg(
    views=("views","sum"),
    orders=("orders","sum"),
    net_revenue_usd=("net_revenue_usd","sum"),
    avg_discount=("discount_rate","mean"),
    ad_spend_usd=("ad_spend_usd","sum"),
    avg_rating=("rating","mean"),
    avg_fulfillment_days=("avg_fulfillment_days","mean"),
).reset_index()
prod_perf["conversion_rate"] = prod_perf["orders"]/prod_perf["views"].replace(0,pd.NA)

high_views_low_orders = prod_perf[
    (prod_perf["views"]>=prod_perf["views"].quantile(0.90)) &
    (prod_perf["orders"]<=prod_perf["orders"].quantile(0.35))
].sort_values(["views","orders"], ascending=[False, True]).head(15)

high_views_low_orders

,product_id,vendor_id,category,sub_category,price_usd,views,orders,net_revenue_usd,avg_discount,ad_spend_usd,avg_rating,avg_fulfillment_days,conversion_rate
161,P00162,V004,Grocery,Pantry,12.47,8398,509,5591.828333,0.011342,161.01,3.91,5.7,0.06061


###AI Component: ML-driven Decision System

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.linear_model import LogisticRegression

clean = pd.read_csv("synthetic_marketplace_daily_clean.csv", parse_dates=["date"])

model_df = clean.sample(80000, random_state=42) if len(clean)>80000 else clean.copy()
model_df["y"] = (model_df["orders"]>0).astype(int)

features = ["category","sub_category","price_usd","discount_rate","ad_spend_usd","rating","rating_count","stock_units","avg_fulfillment_days","vendor_id"]
X = model_df[features]
y = model_df["y"]

cat_features = ["category","sub_category","vendor_id"]
num_features = [c for c in features if c not in cat_features]

pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ("num", "passthrough", num_features)
])

clf = Pipeline([
    ("pre", pre),
    ("lr", LogisticRegression(max_iter=500))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.22, random_state=42, stratify=y)
clf.fit(X_train, y_train)
proba = clf.predict_proba(X_test)[:,1]

print("ROC-AUC:", roc_auc_score(y_test, proba))
print("Avg Precision:", average_precision_score(y_test, proba))


ROC-AUC: 0.8408807894028526
Avg Precision: 0.9962921397588982


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


###Visualization


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

# Load data

clean = pd.read_csv("synthetic_marketplace_daily_clean.csv", parse_dates=["date"])
vendors = pd.read_csv("vendors_master.csv")

# Vendor performance
vendor_perf = clean.groupby("vendor_id").agg(
    views=("views", "sum"),
    orders=("orders", "sum"),
    gross_revenue_usd=("gross_revenue_usd", "sum"),
    net_revenue_usd=("net_revenue_usd", "sum"),
    ad_spend_usd=("ad_spend_usd", "sum"),
    returns=("returns", "sum"),
    avg_discount=("discount_rate", "mean"),
    avg_fulfillment_days=("avg_fulfillment_days", "mean"),
    avg_rating=("rating", "mean"),
).reset_index()

vendor_perf["conversion_rate"] = vendor_perf["orders"] / vendor_perf["views"].replace(0, np.nan)
vendor_perf["return_rate"] = vendor_perf["returns"] / vendor_perf["orders"].replace(0, np.nan)

vendor_perf = vendor_perf.merge(vendors, on="vendor_id", how="left")

top_rev = vendor_perf.sort_values("net_revenue_usd", ascending=False).head(10)
top_conv = vendor_perf[vendor_perf["views"] > 2000].sort_values("conversion_rate", ascending=False).head(10)

# Category performance
cat_perf = clean.groupby("category").agg(
    views=("views", "sum"),
    orders=("orders", "sum"),
    net_revenue_usd=("net_revenue_usd", "sum"),
).reset_index()
cat_perf["conversion_rate"] = cat_perf["orders"] / cat_perf["views"].replace(0, np.nan)
cat_perf = cat_perf.sort_values("net_revenue_usd", ascending=False)

# Product performance (for scatter + later use)
prod_perf = clean.groupby(["product_id", "vendor_id", "category", "sub_category", "price_usd"]).agg(
    views=("views", "sum"),
    orders=("orders", "sum"),
    net_revenue_usd=("net_revenue_usd", "sum"),
    avg_discount=("discount_rate", "mean"),
    ad_spend_usd=("ad_spend_usd", "sum"),
    avg_rating=("rating", "mean"),
    avg_fulfillment_days=("avg_fulfillment_days", "mean"),
    rating_count=("rating_count", "mean"),
    stock_units=("stock_units", "mean"),
).reset_index()
prod_perf["conversion_rate"] = prod_perf["orders"] / prod_perf["views"].replace(0, np.nan)

# Charts (PNGs)
plt.figure(figsize=(9, 5))
plt.bar(top_rev["vendor_id"], top_rev["net_revenue_usd"])
plt.title("Top 10 Vendors by Net Revenue")
plt.xlabel("Vendor")
plt.ylabel("Net Revenue (USD)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("fig_top_vendors_net_revenue.png", dpi=160)
plt.close()

plt.figure(figsize=(9, 5))
plt.bar(top_conv["vendor_id"], top_conv["conversion_rate"])
plt.title("Top 10 Vendors by Conversion Rate (min views threshold)")
plt.xlabel("Vendor")
plt.ylabel("Conversion Rate (orders/views)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("fig_top_vendors_conversion.png", dpi=160)
plt.close()

plt.figure(figsize=(9, 5))
plt.bar(cat_perf["category"], cat_perf["net_revenue_usd"])
plt.title("Net Revenue by Category")
plt.xlabel("Category")
plt.ylabel("Net Revenue (USD)")
plt.xticks(rotation=35, ha="right")
plt.tight_layout()
plt.savefig("fig_category_net_revenue.png", dpi=160)
plt.close()

sample_prod = prod_perf.sample(min(len(prod_perf), 1200), random_state=42).copy()
plt.figure(figsize=(7, 5))
plt.scatter(sample_prod["views"], sample_prod["conversion_rate"].fillna(0), s=10, alpha=0.6)
plt.title("Products: Views vs Conversion Rate (sample)")
plt.xlabel("Total Views")
plt.ylabel("Conversion Rate")
plt.tight_layout()
plt.savefig("fig_views_vs_conversion.png", dpi=160)
plt.close()

# AI component: purchase probability model
# Predict: does a product-day have >=1 order?
model_df = clean.sample(min(len(clean), 80000), random_state=42).copy()
model_df["y"] = (model_df["orders"] > 0).astype(int)

features = [
    "category", "sub_category", "price_usd", "discount_rate", "ad_spend_usd",
    "rating", "rating_count", "stock_units", "avg_fulfillment_days", "vendor_id"
]
X = model_df[features]
y = model_df["y"]

cat_features = ["category", "sub_category", "vendor_id"]
num_features = [c for c in features if c not in cat_features]

pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ("num", "passthrough", num_features)
])

clf = Pipeline([
    ("pre", pre),
    ("lr", LogisticRegression(max_iter=800))
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.22, random_state=42, stratify=y
)
clf.fit(X_train, y_train)
proba = clf.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, proba)
ap = average_precision_score(y_test, proba)
print(f"Model ROC-AUC: {auc:.3f} | Average Precision: {ap:.3f}")

# Recommendations

# Use recent 14 days
recent = clean[clean["date"] >= clean["date"].max() - pd.Timedelta(days=13)].copy()
recent["p_order"] = clf.predict_proba(recent[features])[:, 1]

recent_prod = recent.groupby(["product_id", "vendor_id", "category", "sub_category", "price_usd"]).agg(
    views=("views", "sum"),
    orders=("orders", "sum"),
    p_order=("p_order", "mean"),
    avg_discount=("discount_rate", "mean"),
    stock=("stock_units", "mean"),
    avg_rating=("rating", "mean"),
    avg_fulfillment=("avg_fulfillment_days", "mean"),
).reset_index()
recent_prod["conversion_rate"] = recent_prod["orders"] / recent_prod["views"].replace(0, np.nan)

# Products to discount:
discount_candidates = recent_prod[
    (recent_prod["views"] >= np.percentile(recent_prod["views"], 85)) &
    (recent_prod["p_order"] <= np.percentile(recent_prod["p_order"], 30)) &
    (recent_prod["stock"] > 50) &
    (recent_prod["avg_discount"] < 0.10)
].copy()

discount_candidates["suggested_discount"] = np.clip(
    0.10 + (0.18 - discount_candidates["p_order"]) * 0.6,
    0.10, 0.25
)

discount_candidates = discount_candidates.sort_values(
    ["views", "p_order"], ascending=[False, True]
).head(12)

discount_candidates.to_csv("ai_discount_recommendations.csv", index=False)

# Vendors to promote:
recent_vendor = recent.groupby("vendor_id").agg(
    views=("views", "sum"),
    orders=("orders", "sum"),
    net_rev=("net_revenue_usd", "sum")
).reset_index()
recent_vendor["conversion_rate"] = recent_vendor["orders"] / recent_vendor["views"].replace(0, np.nan)

promo_vendors = recent_vendor[
    (recent_vendor["views"] <= np.percentile(recent_vendor["views"], 40)) &
    (recent_vendor["conversion_rate"] >= np.percentile(recent_vendor["conversion_rate"], 70)) &
    (recent_vendor["orders"] >= 40)
].merge(vendors, on="vendor_id", how="left").sort_values(
    ["conversion_rate", "orders"], ascending=False
).head(8)

promo_vendors.to_csv("ai_vendor_promotion_recommendations.csv", index=False)

print("\nGenerated files:")
print("- fig_top_vendors_net_revenue.png")
print("- fig_top_vendors_conversion.png")
print("- fig_category_net_revenue.png")
print("- fig_views_vs_conversion.png")
print("- ai_discount_recommendations.csv")
print("- ai_vendor_promotion_recommendations.csv")

Model ROC-AUC: 0.851 | Average Precision: 0.996

Generated files:
- fig_top_vendors_net_revenue.png
- fig_top_vendors_conversion.png
- fig_category_net_revenue.png
- fig_views_vs_conversion.png
- ai_discount_recommendations.csv
- ai_vendor_promotion_recommendations.csv


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
